# 4.2 - Office finder mongo version

#### Encontramos tu oficina

In [ ]:
%pip install folium

In [1]:
# librerias

from pymongo import MongoClient

import pandas as pd

import folium   # esto es para hacer mapas

import re

In [2]:
# conexion a mongo

cliente=MongoClient()

db=cliente.companies

colec=db.companies

In [3]:
query={'$and': [
    
    {'$or': [{'category_code': 'games_video'}, 
             {'category_code': 'web'},
             {'category_code': 'ecommerce'}]},
    
    {'founded_year': {'$gte': 1999}},
    {'deadpooled_year': None},
    {'number_of_employees': {'$ne': None}},
    {'number_of_employees': {'$ne': 0}},
    {'total_money_raised': {'$ne': None}}
    
]}


filtro={'name': 1, '_id':0, 'offices.latitude': 1, 'offices.longitude': 1,
         'founded_year': 1, 'category_code':1, 'number_of_employees':1,
         'total_money_raised':1}




data=pd.json_normalize(data=colec.find(query, filtro), 
                       record_path='offices', 
                       meta=['category_code', 'name', 'number_of_employees',
                            'founded_year', 'total_money_raised'])


data.head()

,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,games_video,Kyte,40,2006,$23.4M
4,40.726040,-73.995722,web,Meetup,75,2002,$18.3M


In [4]:
data.insert(2, 'geo_loc', pd.Series(list(range(len(data)))))

In [5]:
# aqui relleno la columna con una lista de longitud y latitud
data['geo_loc']=data.apply(lambda x: [x['longitude'], x['latitude']], axis=1)


data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M


In [6]:
data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M


In [7]:
data=data.dropna()

In [8]:
data.tail()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
2257,29.735462,-95.413243,"[-95.413243, 29.735462]",games_video,Fictionaut,2,2008,$0
2258,40.756054,-73.986951,"[-73.986951, 40.756054]",web,Conversition,10,2008,$0
2260,59.617364,16.559539,"[16.5595393, 59.617364]",web,Gastrogate,12,1999,$0
2261,49.282455,-123.109217,"[-123.109217, 49.2824551]",games_video,Tiny Speck,45,2009,$17.2M
2262,38.752367,-9.146221,"[-9.1462209, 38.7523672]",games_video,My TV Shows,2,2007,$0


In [9]:
def exchange(col):
    
    res=[]
    
    number=1
    
    symbol=1
    
    for e in col:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
        
        n_col=re.findall('[0-9.]+', e)[0]

        
        res.append(int(float(n_col) * number * symbol))
        
    return res

In [10]:
exchange(['$28.3M'])

[28300000]

In [11]:
data['n_total_money']=exchange(data.total_money_raised)

data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000


In [12]:
data[['latitude', 'n_total_money']].head()

,latitude,n_total_money
0,34.090368,16500000
1,37.387845,28300000
2,34.017606,21000000
3,37.788482,23400000
4,40.726040,18300000


In [13]:
data.reset_index(drop=True, inplace=True)

In [14]:
data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000


In [15]:
def classifying(date, employees):
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
        else:
            company_class.append('BigCorp')
            
    
    return company_class

In [16]:
data['company_class']=classifying(data.founded_year, data.number_of_employees)

In [17]:
data.tail()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
1616,29.735462,-95.413243,"[-95.413243, 29.735462]",games_video,Fictionaut,2,2008,$0,0,StartUp
1617,40.756054,-73.986951,"[-73.986951, 40.756054]",web,Conversition,10,2008,$0,0,StartUp
1618,59.617364,16.559539,"[16.5595393, 59.617364]",web,Gastrogate,12,1999,$0,0,BigCorp
1619,49.282455,-123.109217,"[-123.109217, 49.2824551]",games_video,Tiny Speck,45,2009,$17.2M,19264000,StartUp
1620,38.752367,-9.146221,"[-9.1462209, 38.7523672]",games_video,My TV Shows,2,2007,$0,0,BigCorp


In [18]:
# insertar en mongo

#df.to_json('geoloc.json', orient='records', lines=True)

In [19]:
data=data.dropna()

In [20]:
db.geoloc.insert_many(data.to_dict('records'))

In [21]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [22]:
# geolocalizacion

def geoquery(df):
    
    list_money_mean=[]
    
    sb_ratio=[]
    
    n_companies=[]
    
    for i in range(len(df)):
        
        q=list(db.geoloc.find({'geo_loc': {'$near': {'$geometry': {
                                     
                                'type': 'Point',
                                'coordinates': df.geo_loc[i]},
                                
                                '$maxDistance': 1000}}}))
        
        
        long_list=len(q)
        
        # media de dinero
        money_mean=sum([q[j]['n_total_money'] for j in range(long_list)]) / long_list
        
        list_money_mean.append(money_mean)
        
        # startup-bigcorp
        
        big=sum(q[k]['company_class'].count('BigCorp') 
                if q[k]['company_class'].count('BigCorp')!=0 
                else 1 for k in range(long_list))
        
        start=sum(q[k]['company_class'].count('StartUp')  for k in range(long_list))
        
        # si sb_ratio=1 entonces big=start, si sb_ratio<1 hay más big, si sb_ratio>1 hay mas start
        sb_ratio.append(start/big)
        
        
        # nº compañias
        n_companies.append(long_list)
        
        
    df['around_companies']=n_companies
    
    df['sb_ratio']=sb_ratio
    
    df['money_density']=list_money_mean
    
    return df

In [23]:
data=geoquery(data)

data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,sb_ratio,money_density
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp,2,0.000000,1.650000e+07
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp,4,0.000000,1.415000e+07
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp,6,0.333333,8.333333e+06
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000,BigCorp,48,0.375000,8.157333e+06
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000,BigCorp,32,0.187500,4.370625e+06


In [24]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                    tiles='openstreetmap', 
                    zoom_start=zoom_start)
    
    
    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], 
                            popup=str(df.name[i]), 
                            radius=1, 
                            icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    return mapa

In [25]:
print('oficinas')

mapa=geomap(data.loc[:1500])

mapa

oficinas


In [26]:
print ('Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$')

df=data[(data.around_companies >= 10) & (data.sb_ratio > 0.45) & (data.money_density > 1e6)]

mapa=geomap(df)

mapa

Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$


In [27]:
print ('Compañias alrededor>=10, sb_ratio>0.5 y densidad monetaria>100.000$')

df=data[(data.around_companies >= 10) & (data.sb_ratio > 0.5) & (data.money_density > 1e5)]

mapa=geomap(df)

mapa

Compañias alrededor>=10, sb_ratio>0.5 y densidad monetaria>100.000$
